In [1]:
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [2]:
test_tensor = np.random.rand(5000).reshape(10, 10, 10, 5)

In [8]:
test_tensor = tf.random.uniform(
    shape=(10, 100, 100, 5), minval=0, maxval=1
)

In [12]:
len(test_tensor.shape)

4

In [ ]:
def augment(image, label):
    batch_data = tf.concat([image, label])
    if batch_data.shape.size > 3
        angles = tf.random
    rotated = tfa.image.rotate(batch_data, angles=)